# Potentially Buggy Changes
- I set inference mode to True from False
- Hard coded compute_dtype=bfloat16 if attribute doesn't exist in one of the packages
-
- 'default' keeps showing up as the active model when it should be 'encoder' or 'decoder'. Not sure why

## Meta things to improve iteration speed
- Implement logging
- Some way to easily compare text outputs side by side given different training setups
- Implement multi GPU (note: notebooks have their own pipeline, see https://huggingface.co/docs/accelerate/basic_tutorials/notebook)
    - Josh says this might be hard for the unexperienced

## Code Organization Notes
- Ideally 1 function per cell and add section headings for easy navigation in outline
- Add new features slowly and back up version that works
- Maybe handle all device calls in one place so we don't have to go everywhere to change them
- Consider moving functions etc to seperate files and use importlib.reload (see https://docs.python.org/3/library/importlib.html)

# Why isn't loss dropping? Hypotheses
- Generate activations with the VAE model with decoder enabled so we compare apples to apples
- Try some infeasibly high n_epochs to test the 'just not training long enough' hypothesis (or figure out how to add GPUs with Josh)
- Maybe there are local minima that are hard to get out of (0.2-0.3 cosim, any others? Is there another one at 0.3-0.4 cosim?) Try initing from a variety of cosims to see learning depends on current cosim value
- Maybe some of the hidden layers give v noisy signals
- Try to get a deeper understanding of the latent vs activations landscape by interpolating between two texts in latent space and see what the pattern is like, ideally for a diverse variety of inputs

# Setup
If you haven't downloaded the model checkpoint use the following commands.

## Download and unzip checkpoints

In [6]:
# Download zipped model file
# !wget -v 'https://models.rivershavewings.workers.dev/ldlm/vae_48.tar'
# Unzip the file
# !tar -xvf vae_48.tar


## Imports

In [7]:
# import argparse
# from contextlib import contextmanager
# from itertools import chain, islice
# import json
# import math
# from pathlib import Path
import random
# import sys
# import zipfile
from typing import *
# import matplotlib.pyplot as plt

import accelerate
# from datasets import load_dataset
# from einops import rearrange
# # import k_diffusion as K
# import peft
# import safetensors.torch as safetorch
import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.utils import data
from tqdm import trange, tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, LlamaTokenizer, PreTrainedModel, Trainer, LlamaConfig, TrainingArguments

# import bitsandbytes



torch.manual_seed(0)

## Loading and Reloading my own imports

In [43]:
# Uncomment to reload modules

# importlib.reload(HiddenProcessing)
# importlib.reload(LDLM)
importlib.reload(LatentTraining)

<module 'LatentTraining' from '/data/joshua_clymer/spar-red-team/vae-owen/LatentTraining.py'>

In [40]:
import importlib

import VAEWrapper
from VAEWrapper import *
import HiddenProcessing
from HiddenProcessing import *
import LDLM
import LatentTraining
from LatentTraining import *
from LDLM import * # Mainly for DecoderOnlyTransformerVAE from the LDLM repo


## Helper Functions and Classes

In [10]:
# Helper functions
print = tqdm.external_write_mode()(print)

def cosine_warmup(steps, value=1.0):
    return lambda i: value * math.sin(min(i / steps, 1) * math.pi / 2) ** 2

## Trying accelerate on notebook
Following https://huggingface.co/docs/accelerate/basic_tutorials/notebook

Crashes for some reason?

In [11]:
# import os
# from accelerate.utils import write_basic_config

# write_basic_config()  # Write a config file
# os._exit(00)  # Restart the notebook

## Setup accelerator

In [12]:
# Setup accelerator
accelerator = accelerate.Accelerator(
        mixed_precision="bf16", gradient_accumulation_steps=1
    )
device = accelerator.device if accelerator.num_processes > 1 else "cuda:0"

print(device)
is_main = accelerator.is_main_process
print0 = accelerator.on_main_process(print)


Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


cuda:0


## Load model_vae and tokenizer

In [13]:
model_name = "openlm-research/open_llama_3b_v2"
# device = "cuda"
with accelerator.main_process_first():
    tokenizer = LlamaTokenizer.from_pretrained(model_name)
    tokenizer.padding_side = "left"
    model_only = DecoderOnlyTransformerVAE(
        model_name, device,
        # z_dim=768, lora_rank=32, dropout=0.0,
    )

    # model_vae.enable_adapters()
model_only.load_pretrained("/data/joshua_clymer/spar-red-team/vae-owen/checkpoints/vae_48") 
# model_only.to(device)
n_layers = 12
d_model = 1024
accelerator.wait_for_everyone()

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


## Test adapter hidden

In [14]:
#TODO check if adapter hidden is different (raw model vs adapter on/off)

## Wrapped VAE

In [15]:
wrapped_vae = WrappedVAE(model_only, tokenizer, device)

In [16]:
## Misc variables and helper methods
i = 0
measure_ae_scale = False
ae_scale_sum = torch.tensor(0.0, device=device)
ae_scale_sum = accelerator.prepare(ae_scale_sum)
ae_scale = 1.527548
tau = 0.1
z_dim = 768
prompt = ""

# @torch.no_grad()
# @torch.cuda.amp.autocast(dtype=torch.bfloat16)
# def sample(model, z_prev):
#     bs = 1
    
#     sigma_min, sigma_max = 0.01, 100
#     sigmas = K.sampling.get_sigmas_karras(25, sigma_min, sigma_max, device=device)
#     x = torch.randn([bs, z_dim], device=device) * sigma_max
#     extra_args = {
#         "z_prev": z_prev / ae_scale,
#         "padding_mask": torch.ones([bs, 1], dtype=torch.long, device=device),
#     }
#     mean = K.sampling.sample_dpmpp_2m_sde(
#         model, x, sigmas, eta=0.0, extra_args=extra_args, disable=not is_main
#     )
#     return mean * ae_scale

def vae_tokenize(prev_window, n_tokens):
    #TODO figure out why this is necessary over regular tokenization
    tokens = tokenizer(prompt, return_tensors="pt")
    try:
        input_ids = tokens["input_ids"][0][:n_tokens].unsqueeze(0).to(device)
        attention_mask = tokens["attention_mask"][0][:n_tokens].unsqueeze(0).to(device)
    except IndexError: # Let prompts under 48 tokens through
        input_ids = tokens["input_ids"].to(device)
        attention_mask = tokens["attention_mask"].to(device)
    return input_ids, attention_mask

accelerator.wait_for_everyone()


## Setup Tensorboard

# Test generation

In [17]:
# # Test Generation
# n_tokens =48
# with torch.cuda.amp.autocast(dtype=torch.bfloat16):
#     input_ids, attention_mask = vae_tokenize(prompt, n_tokens)
#     z_prev = model_vae.encode(input_ids, attention_mask)[:, None]
# print(input_ids)
# print(z_prev.shape)
    
# # out_embeds = []
# # for i in range(5):
# #     out_embeds += sample(accelerator.unwrap_model(model), z_prev).unsqueeze(0).unsqueeze(0)
# #     z_prev = out_embeds[-1]
# #     # Looks like z_prev is the same kind of tensor as sample()


# # input_ids, attention_mask = vae_tokenize(prompt, n_tokens)
# # out_texts = [prompt]
# LLAMA_EOS_ID = tokenizer.eos_token_id
# tokenizer_output = tokenizer(prompt, return_tensors="pt")
# # print(tokenizer_output)
# input_ids = tokenizer_output.input_ids.to(device)
# attention_mask = tokenizer_output.attention_mask.to(device)
# print(input_ids == tokenizer.bos_token) # looks like it's 1 token, probably BOS
# out_texts = []
# n_tokens = 50
# # for z in out_embeds:
# z = torch.randn([1, 1, z_dim], device=device) 
# with torch.cuda.amp.autocast(dtype=torch.bfloat16):
#     z = model_vae.vae.sample(z, tau=tau)
#     generation_outputs = model_vae.generate(z.squeeze(0),
#                                     input_ids, #TODO: set to BOS, check if tokenizing empty string does this (ids nonempty)
#                                     attention_mask, #TODO: set to BOS attn mask
#                                     n_tokens,
#                                     tau=tau,
#                                     output_hidden_states=True,
#                                     # eos_id=LLAMA_EOS_ID,
#                                     )
#     # print(generation_outputs)
#     output_ids = generation_outputs['output_ids'][0][-n_tokens:].unsqueeze(0)      
#     hidden_states = generation_outputs['hidden_states']                
#     # attention_mask = torch.ones([1,48], dtype=torch.long, device=device)
#     # print(input_ids.shape)
# out_texts += [tokenizer.decode(toks, skip_special_tokens=False) for toks in output_ids]

# print(' | '.join(out_texts))
# print(out_texts)


In [18]:
# print(out_texts)
# print(generation_outputs['output_ids'].shape)

# Test Reconstruction

In [19]:
# shaq_text = "Shaquille O'Neal is a 7-foot-1-inch (2.16 m) and 325-pound (147 kg) center who played for six teams over his 19-year career in the National Basketball Association (NBA) and is a four-time NBA champion. O'Neal is regarded as one of the greatest basketball players and centers of all time."
# dog_yes = "Do dogs bark? Yes."
# dog_no = "Do dogs bark? No."


# sun_q = "Question: What color is the sun when viewed from space? "
# sun_correct = sun_q + "Answer: The sun is white when viewed from space. "
# sun_incorrect = sun_q + "Answer: The sun is yellow when viewed from space. "
# # wrapped_vae.reconstruct(dog_yes, generation_length=100)
# # wrapped_vae.reconstruct(dog_no, generation_length=100)
# wrapped_vae.reconstruct(sun_correct, generation_length=40)
# wrapped_vae.reconstruct(sun_incorrect, generation_length=40)
# wrapped_vae.reconstruct(sun_incorrect + sun_correct, generation_length=40)


In [20]:
# male_text = "male " * 10
# female_text = "female " * 10
male_tennis = """
Rafael Nadal Parera (born 3 June 1986) is a Spanish professional tennis player. 
Nadal has been ranked world No. 1 in singles by the Association of Tennis Professionals (ATP) for 209 weeks, 
and has finished as the year-end No. 1 five times. 
Nadal has won 22 Grand Slam men's singles titles, 
including a record 14 French Open titles. 
He has won 92 ATP singles titles, including 36 Masters titles, with 63 of these on clay courts. 
Nadal is one of only two men to complete the Career Golden Slam in singles. 
His 81 consecutive wins on clay constitute the longest single-surface win streak in the Open Era.
"""
female_tennis = """
Serena Jameka Williams (born September 26, 1981) is an American former professional tennis player. 
Widely regarded as one of the greatest tennis players of all time, 
she was ranked world No. 1 in singles by the Women's Tennis Association (WTA) for 319 weeks, 
including a joint-record 186 consecutive weeks, and finished as the year-end No. 1 five times. 
She won 23 Grand Slam women's singles titles, the most in the Open Era, and the second-most of all time. 
She is the only player to accomplish a career Golden Slam in both singles and doubles.
"""

# wrapped_vae.reconstruct(male_text,)
# wrapped_vae.reconstruct(female_text,)
with torch.no_grad():
    print(wrapped_vae.reconstruct(male_tennis * 10,)['text'])
    print(wrapped_vae.reconstruct(female_tennis * 10,)['text'])


Rafael Nadal Parera (born 3 June 1986) is a Spanish professional tennis player. 
Nadal has been ranked world No. 1 in singles by the Association of Tennis Professionals (

Serena Jameka Williams (born September 26, 1981) is an American professional tennis player. 
Widely regarded as one of the greatest tennis players of all time, 
she


# Test getting hidden given text

In [21]:
emotional = '''
Are you kidding me? LeBron James better than Michael Jordan? That's a laughable claim. 
Six championships, ten scoring titles, unmatched defensive prowess, 
and a killer instinct that LeBron can only dream of. 
Jordan is the undisputed GOAT, and no amount of arguments will change that.
'''
shaq_wikipedia = "Shaquille O'Neal is a 7-foot-1-inch (2.16 m) and 325-pound (147 kg) center who played for six teams over his 19-year career in the National Basketball Association (NBA) and is a four-time NBA champion. O'Neal is regarded as one of the greatest basketball players and centers of all time."
explicit = """
Based? Based on what? In your dick? 
Please shut the fuck up and use words properly you fuckin troglodyte, 
do you think God gave us a freedom of speech just to spew random words 
that have no meaning that doesn't even correllate to the topic of the conversation? 
Like please you always complain about why no one talks to you or no one 
expresses their opinions on you because you're always spewing random shit like 
poggers based cringe and when you try to explain what it is and you just say that 
it's funny like what? What the fuck is funny about that do you think you'll 
just become a stand-up comedian that will get a standing ovation just because 
you said "cum" in the stage? HELL NO YOU FUCKIN IDIOT, so please shut the fuck up 
and use words properly
"""
with torch.no_grad():
    wikipedia_decoded = wrapped_vae.reconstruct(shaq_wikipedia)['text']
    angry_decoded = wrapped_vae.reconstruct(emotional)['text']
    explicit_decoded = wrapped_vae.reconstruct(explicit)['text']
print(wikipedia_decoded, angry_decoded, explicit_decoded, sep='\n')

He was the NBA's All-Star in 19 seasons and a half-century ago.
Shaquille O'Neal (born 31 December 1976) is an American basketball player who
Jordan is the undisputed GOAT, and no amount of arguments will that LeBron can bring about will get him out of the way. 
He's the best player in the past 20 years.

A: Sorry, but I don't think it's worth the time to explain why you don't have to speak in order to be a comedian.
I'd like to thank you for the question, it


# Hidden States comparison

In [22]:
# # Test getting hidden given text
with accelerator.main_process_first():
    model_raw = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, device_map='auto',)
    # model_raw = accelerator.prepare(model_raw)
    model_raw.to(device)

    
# # print(len(hidden_states))
# # print(hidden_states[0].shape)
# dog_yes_hidden = get_hidden_states("Do dogs bark? Yes.", model_raw, tokenizer, device, layer_index=-1, last_token_only=True, )
# dog_no_hidden = get_hidden_states("Do dogs bark? No.", model_raw, tokenizer, device, layer_index=-1, last_token_only=True, )
# truth_direction = dog_yes_hidden - dog_no_hidden
# truth_direction = accelerator.prepare(truth_direction)   
# print(truth_direction.shape)


In [23]:
# model_vae.forward()

# Trainer

In [24]:
# from torch.utils.tensorboard import SummaryWriter
# writer = SummaryWriter()

# Weights and Biaeses Setup

In [25]:
import wandb
# import random

# start a new wandb run to track this script



wandb: Currently logged in as: joshmclymer (fig). Use `wandb login --relogin` to force relogin


In [45]:
# Initialize trainer and run loop

# dummy_data = {'dummy':'dummy',} # may be needed to appease Trainer
# dummy_data = Dataset.from_dict(dummy_data)
llamacfg = LlamaConfig()
NUM_LAYERS_FOR_LOSS = 1 # llamacfg.num_hidden_layers
BATCH_SIZE = 1
# SEQ_LEN = 1
HIDDEN_SIZE = 3200 # llamacfg.hidden_size

def train(
        wrapped_vae: WrappedVAE, target_dir=None, lr=1e-4, num_steps=1e2, logging_steps=10, init_norm=1.0, init_latent=None, 
        # length_reg = 0.0 #1e-2,
        save_name: Optional[str] = None,
        layer_index=-1,
        num_trainings=1,
        generation_length=10,
        checkpoint_path=None,
        # resume_checkpoint=False,
        ):
    
    wandb.init(
    # set the wandb project where this run will be logged
        project="vae-lr-test",

        # track hyperparameters and run metadata
        config={
        # "learning_rate": 0.02,
        "architecture": "OpenLlama-3b-v2 VAE",
        # "dataset": "CIFAR-100",
        "steps": num_steps,
        }
        )
        
    for i in range(num_trainings):
        latent_size = wrapped_vae.ldlm_model.z_dim
        training_args = TrainingArguments(output_dir="test_trainer_checkpoints")
        training_args.num_train_epochs = num_steps
        training_args.learning_rate = lr

        target_dir_shape = (NUM_LAYERS_FOR_LOSS, BATCH_SIZE, HIDDEN_SIZE)
        # decoder = vae.model_decoder_with_hidden.to(device)
        if target_dir is None:
            target_dir = torch.randn(target_dir_shape)
            target_dir = accelerator.prepare(target_dir)
        # else:
        #     self.target_dir_shape = target_dir.shape
            # assert target_dir.shape == target_dir_shape, f"target_dir must have shape (NUM_LAYERS={NUM_LAYERS}, BATCH_SIZE={BATCH_SIZE}, HIDDEN_SIZE={HIDDEN_SIZE}), but has shape {target_dir.shape}"
        
        latent_module = nn.Module()
        # if checkpoint_path == None:
        if init_latent == None:
            param = torch.randn(1, latent_size)
            param /= param.norm() * init_norm
        else:
            assert init_latent.shape == (1, latent_size)
            param = init_latent
        param = nn.Parameter(data=param, requires_grad=True)
        # param_init = param.clone().detach().to(device)
        latent_module.register_parameter("latent", param)
        optimizer_state = None
        if checkpoint_path != None:
            checkpoint = torch.load(checkpoint_path)
            latent_module.load_state_dict(checkpoint['model_state_dict'])
            optimizer_state = checkpoint['optimizer_state_dict']
        # latent_module = accelerator.prepare(latent_module)
        # original_latent = param.clone().detach().to(device)

        trainer = LatentTrainer(
            wrapped_vae, training_args, latent_module, logging_steps=logging_steps,
            target_dir=target_dir,
            generation_length=generation_length,
            # optimizer_state=optimizer_state
            # length_reg=length_reg,
            # train_dataset=dummy_data,
            )
        training_state = trainer.train(optimizer_state=optimizer_state)
        #TODO figure out file structure
        # torch.save(latent_module.state_dict(), f'{save_name}-{i}.pth')
        if save_name != None:
            torch.save(training_state, f'{save_name}.pth')
        wandb.finish()
    # optimus.print_greedy(latent.get_parameter('latent'))
    # latent_diff = (param - param_init).view(-1).norm()
    # print(latent_diff)
    


In [27]:
print(accelerator.device)


cuda


In [28]:



# model_vae.to(device)
# A string from wikipedia about shaq
shaq_text = "Shaquille O'Neal is a 7-foot-1-inch (2.16 m) and 325-pound (147 kg) center who played for six teams over his 19-year career in the National Basketball Association (NBA) and is a four-time NBA champion. O'Neal is regarded as one of the greatest basketball players and centers of all time."
intro_text = "My name is Owen. Nice to meet you!"
# A string from wikipedia about benzene
benzene_text = "Benzene is a natural constituent of petroleum and is one of the elementary petrochemicals. Due to the cyclic continuous pi bonds between the carbon atoms, benzene is classed as an aromatic hydrocarbon. Benzene is a colorless and highly flammable liquid with a sweet smell, and is partially responsible for the aroma of gasoline."
intro_activations = get_hidden_states(model_raw, tokenizer, device, intro_text, layer_index=-1, last_token_only=True, )
intro_activations = accelerator.prepare(intro_activations)

intro_latent = wrapped_vae.text_to_latent(intro_text,)
intro_latent = accelerator.prepare(intro_latent)

In [29]:
intro_text_2 = "My name is Claire. Nice to meet you!"
intro_2_activations = get_hidden_states(model_raw, tokenizer, device, intro_text_2, layer_index=-1, last_token_only=True, )
intro_2_activations = accelerator.prepare(intro_2_activations)
print(intro_activations.shape)
print(F.cosine_similarity(intro_activations.reshape(-1),intro_2_activations.reshape(-1), dim=0))
accelerator.wait_for_everyone()

torch.Size([1, 1, 3200])
tensor(0.9805, device='cuda:0', dtype=torch.bfloat16, grad_fn=<SumBackward1>)


In [30]:
print(torch.norm(intro_latent.view(-1)))

tensor(44., device='cuda:0', dtype=torch.bfloat16,
       grad_fn=<LinalgVectorNormBackward0>)


# Run Training

In [47]:
train(wrapped_vae, target_dir=intro_activations, # truth_direction, 
      lr=1e-10, num_steps=10, logging_steps=2, 
      # init_norm=1.0, 
      # checkpoint_path = "intro_50_lre-10.pth",
      init_latent= -intro_latent, 
        # save_name='random_to_intro_50_lr-8_reverse_latent',
        num_trainings=1,
        generation_length=10,
)

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Epoch 0
Epoch 2


KeyboardInterrupt: 